In [36]:
from collections import deque
from imutils.video import VideoStream
import numpy as np
import argparse
import cv2
import imutils
import time

# Path to the video file
video_path = r"C:\Users\sarth\Downloads\WHATSAAP ASSIGNMENT.mp4"
# HSV color range for detecting yellow
yellowLower = (20, 100, 100)
yellowUpper = (30, 255, 255)
# Deque to store the tracked points
pts = deque()

# Initialize the video stream and background subtractor
vs = cv2.VideoCapture(video_path)
bg_subtractor = cv2.createBackgroundSubtractorMOG2()

# Wait for the camera to warm up
time.sleep(2.0)

# Variables for dribble detection
dribble_count = 0
is_dribbling = False
dribble_frame_count = 0
dribble_threshold = 2
dribble_speeds = []
prev_center = None

# Main loop to process each frame of the video
while True:
    ret, frame = vs.read()
    if not ret:
        break
    fg_mask = bg_subtractor.apply(frame)

    # Resize the frame and apply Gaussian blur
    frame = imutils.resize(frame, width=500)
    blurred = cv2.GaussianBlur(frame, (11, 11), 0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)

    # Create a mask for yellow objects
    mask = cv2.inRange(hsv, yellowLower, yellowUpper)
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)

    # Find contours in the mask
    contours, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    center = None

    if len(contours) > 0:
        # Find the largest contour and its centroid
        c = max(contours, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))

        if radius > 10:
            # Draw the circle and centroid on the frame
            cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)
            cv2.circle(frame, center, 5, (0, 0, 255), -1)

            # Check for dribble detection
            if prev_center is not None:
                if abs(center[0] - prev_center[0]) > 49 or abs(center[1] - prev_center[1]) > 49:
                    dribble_frame_count += 1
                    if dribble_frame_count >= dribble_threshold:
                        if not is_dribbling:
                            dribble_count += 1
                            is_dribbling = True
                            start_time = time.time()
                    else:
                        is_dribbling = False
                else:
                    dribble_frame_count = 0
                    if is_dribbling:
                        end_time = time.time()
                        time_taken = end_time - start_time
                        if time_taken > 0:
                            distance = np.sqrt((center[0] - prev_center[0])**2 + (center[1] - prev_center[1])**2)
                            dribble_speed = (distance / time_taken) / 1000
                            dribble_speeds.append(dribble_speed)
                        is_dribbling = False

            prev_center = center

    pts.appendleft(center)

    # Display dribble count on the frame
    cv2.putText(frame, f'Dribbles: {dribble_count}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
    # Show the frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # Check for quit key
    if key == ord("q"):
        break

# Release the video stream and close all windows
vs.release()
cv2.destroyAllWindows()

# Print the total dribble count and average dribble speed
print('Total Dribbles: ', dribble_count)
if dribble_speeds:
    print('Average Dribble Speed: ', sum(dribble_speeds) / len(dribble_speeds))
else:
    print('No dribble speeds recorded.')

Total Dribbles:  105
Average Dribble Speed:  1.2308768676504342
